<img src="http://www.finanze.gov.it/export/system/modules/it.gov.finanze.template/resources/img/logo-mef.gif"  style="float: left; width: 25%; margin-left: 8%; margin-top: 0.5em;">

<img src="http://www.opencoesione.gov.it/static/img/header/testata.png" style="float: right; width: 42%; margin-right: 8%; margin-top: 1.6em;">

<p style="clear: both;">

# Studio a livello comunale sui finanziamenti a enti privati da dati OpenCoesione

### Obiettivi 
Questo studio si propone come esempio delle seguenti opportunita' offerte dal Data Analytics Framework (DAF):
 - Integrare dati da fonti diverse permette di compiere analisi piu' complete e semplici
 - L'uso di strumenti open-source consente di impiegare le piu' recenti ed avanzate tecniche di analisi ...

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import gc #garbage collector
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,12)

## Caricamento dataset da integrare

In [3]:
# --- Spese e redditi da OpenCivitas
speseERedditiComune = pd.read_csv('../opencivitas/data/opencivitas_spesa_storica_dati_irpef.csv') #dati per anno

provToRegioni = pd.read_csv('../provToReg.csv', dtype='str', index_col='Sigla automobilistica')
speseERedditiComune = speseERedditiComune[~speseERedditiComune['Codice Istat Comune'].isna()]
speseERedditiComune = pd.merge(
    speseERedditiComune, provToRegioni, how='left', left_on='Sigla Provincia_x',right_index=True)

padZeros = np.vectorize(lambda x: str(x).rjust(6, '0')) # some utils

speseERedditiComune['Codice Istat Lungo'] = \
(speseERedditiComune['Codice regione'] + padZeros(speseERedditiComune['Codice Istat Comune'].astype(int))).astype(int)

# --- Soggetti e luoghi da OpenCoesione
soggettiLuoghi = pd.read_csv('../pac_opencoesione/SoggettiELuoghi.csv', low_memory=False)
soggettiLuoghi.drop(columns=['SOGG_COD_RUOLO', 'SOGG_PROGR_RUOLO'], inplace=True)
soggettiLuoghi = soggettiLuoghi[~soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.isna()]
assert not any(soggettiLuoghi['COD_COMUNE_SEDE_SOGG'].isna()), 'Unexpected missing codes'
soggettiLuoghi['COD_COMUNE_SEDE_SOGG'] = soggettiLuoghi['COD_COMUNE_SEDE_SOGG'].astype(int)
soggettiLuoghi['COD_COMUNE_LUOGO'] = soggettiLuoghi['COD_COMUNE_LUOGO'].fillna(value=-1).astype(int)

## Integrazione OpenCoesione and OpenCivitas

In [11]:
# --- Anagrafica Comuni
path_comuni = '../pac_agenziaentrate/regioni_citta/comuni/archivio_comuni/ElencoComuniAttuali_20170918.csv.read'

comuniAnagrafica = pd.read_csv(path_comuni, sep=';')
comuniAnagrafica.set_index('Codice Istat', inplace=True)
comuniAnagrafica = comuniAnagrafica[~comuniAnagrafica.index.isnull()]
assert not any(comuniAnagrafica.index.duplicated()), 'Duplicate Istat codes'
convTable = pd.read_csv('../provToReg.csv', index_col='Sigla automobilistica', dtype='str')
comuniAnagrafica = comuniAnagrafica.join(convTable, on='Sigla Provincia')
formattedValues = comuniAnagrafica.index.values.astype('int')
padZeros = np.vectorize(lambda x: str(x).rjust(6, '0'))
comuniAnagrafica['Codice Istat Lungo'] = (comuniAnagrafica['Codice regione'] + padZeros(formattedValues)).astype(int)

#### Estrazione fondi erogati a soggetti privati per cui la sede e' riferibile ad un comune:

In [15]:
bEntePrivato = soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.str.startswith('1') & \
                    (soggettiLuoghi.COD_COMUNE_SEDE_SOGG != -1)
soggLuoghiPrivato = soggettiLuoghi[bEntePrivato]

# luoghi (comuni) di sede soggetto in ordine di fondi ricevuti
fondiSoggetti = soggLuoghiPrivato.groupby(
                    'COD_COMUNE_SEDE_SOGG')['TOT_PAGAMENTI'].sum().sort_values(ascending=False).to_frame()

print(fondiSoggetti.head(10))
print(fondiSoggetti.sum())

                      TOT_PAGAMENTI
COD_COMUNE_SEDE_SOGG               
12058091               6.926394e+09
3015146                8.531263e+08
15063049               3.882678e+08
19082053               3.485534e+08
18078045               3.253154e+08
16072006               3.067115e+08
16071024               2.988484e+08
9048017                2.979771e+08
7010025                2.316785e+08
15064065               2.000012e+08
TOT_PAGAMENTI    1.359484e+10
dtype: float64


#### In OpenCivitas non vi sono dati per tutti i Comuni, controlliamo quanto e' grande la quota di fondi in OpenCoesione riferita a sedi soggetto mancanti in OpenCivitas

In [24]:
#comuneCodesPrivato = soggLuoghiPrivato.COD_COMUNE_SEDE_SOGG.drop_duplicates()
#mismatching = comuneCodesPrivato[(comuneCodesPrivato.isin(comuniAnagrafica['Codice Istat Lungo'])) & ~(
        #comuneCodesPrivato.isin(speseERedditiComune['Codice Istat Lungo']))]


newData = fondiSoggetti.join(comuniAnagrafica.set_index('Codice Istat Lungo'))

listaComuniOpenCiv = speseERedditiComune['Codice Istat Lungo'].drop_duplicates()

#print(listaComuniLeft.isin(fondiRicevuti.index).mean(), comuniAnagrafica['Codice Istat Lungo'].isin(fondiRicevuti.index).mean())
#print( fondiRicevuti.index.drop_duplicates().isin(listaComuniLeft).mean(), comuniAnagrafica['Codice Istat Lungo'].isin(listaComuniLeft).mean())

#print( listaComuniLeft.isin(comuniAnagrafica['Codice Istat Lungo']).mean())
#print( fondiRicevuti.index.isin(comuniAnagrafica['Codice Istat Lungo']).mean())


print(fondiSoggetti.sum())

print(fondiSoggetti[~fondiSoggetti.index.isin(listaComuniOpenCiv.values)].sum())

unioneComuniSoggetti = pd.merge(speseERedditiComune, fondiSoggetti,
                                how='left',left_on='Codice Istat Lungo', right_index = True)

#fondiRicevuti[fondiRicevuti.index.isin(listaComuniLeft)]

TOT_PAGAMENTI    1.359484e+10
dtype: float64
TOT_PAGAMENTI    1.378164e+09
dtype: float64


In [28]:



#unioneComuniSoggetti.TOT_PAGAMENTI.fillna(value=0, inplace=True)
#print(newData.COD_COMUNE_SEDE_SOGG.drop_duplicates().isin(speseERedditiComune['Codice Istat Lungo']).mean())


unioneComuniSoggetti.ANNO.unique()



array([2012, 2013])

In [ ]:
#newData.sort_values('TOT_PAGAMENTI', ascending=False)

ax = plt.scatter(np.log10(unioneComuniSoggetti.SPESA_CCC_tot), np.log10(unioneComuniSoggetti.TOT_PAGAMENTI+1))
plt.show()


In [ ]:
bDodgyDataPublic = soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.str.startswith('2') & (soggettiLuoghi.COD_COMUNE_SEDE_SOGG != -1)
#soggettiLuoghi[bDodgyDataPublic].sort_values('TOT_PAGAMENTI', ascending=False)
#soggettiLuoghi['TOT_PAGAMENTI'][bDodgyData].sort_values(ascending=False)
soggettiLuoghi[bDodgyDataPublic].DESCR_FORMA_GIURIDICA_SOGG.unique()

In [ ]:
bDodgyData = soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.str.startswith('1') & (soggettiLuoghi.COD_COMUNE_SEDE_SOGG == -1)
soggettiLuoghi[bDodgyData].sort_values('TOT_PAGAMENTI', ascending=False)
soggettiLuoghi[bDodgyData].DESCR_FORMA_GIURIDICA_SOGG.unique()